In [67]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import sys
import os
import re
from collections import Counter
from pprint import pprint
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import copy
import pickle
import re
import math

In [68]:
# Download the stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/myasin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [69]:
# Import the csv file with all downloaded READMES
df = pd.read_csv("../user_data_csv/csv_readme_per_user.csv")
df.head(10)

,USERNAMES,READMES
0,alexbw,Sampling Inference for Bayesian HSMMs and HMMs...
1,iassael,DEARanking \n Proposing a hybrid DEA/Polynomia...
2,bshillingford,id3tag-fix codehackathon2014 \n Team members:\...
3,hmansell,NaN
4,clementfarabet,sys \n Has moved to a more community friendly ...
5,ajabri,pytorch-maml \n This is a PyTorch implementati...
6,USERNAMES,READMES
7,coryf,homebrew \n Personal homebrew formula Sudoku C...
8,jrodrigomg,"SERVIDOR NODEJS PROYECTO 1, SISTEMAS OPERATIVO..."
9,vascofernandes,CustomersVRPF \n The project consists of two m...


In [70]:
# Apply different methods to clean the text e.g. remove appestorph and unnessary signs
def clean_df(df):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    df['READMES'] = df['READMES'].str.replace('[^a-zA-Z0-9]', ' ', regex=True).str.strip()
    df["READMES"] = df["READMES"].apply(str)
    df["READMES"] = df["READMES"].str.replace(r"[\"\',]", '')
    df["READMES"] = df["READMES"].apply(lambda x: ''.join(ch for ch in x if ch not in set(symbols)))
    df['READMES'] = df['READMES'].str.replace('\d+', '') 
    df['READMES'] = df['READMES'].str.lower()
    df = df.dropna(how='all')
    return df


In [71]:
# Remove punctiation
def remove_punctuation(df):
    df["READMES"] = df['READMES'].str.replace('[^\w\s]','')
    return df

In [72]:
# Remvoe all stopwords from the READMES column
def remove_stop_words(df):
    # import the stopwords from the nltk library
    stop_words = stopwords.words('english')
    df["READMES"] = df["READMES"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    return df

In [73]:
# Remove all single characters from the READMES column
def remove_single_characters(df):
    df['READMES'] = df['READMES'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
    return df

In [74]:
# # Stemming the READMES form the dataframe column
# def stemming(df):
#     stemmer= PorterStemmer()
#     df['READMES'] = df['READMES'].apply(lambda x: [stemmer.stem(y) for y in x])
#     return df

In [75]:
# Call all function to clean and preprosses the READMES
def preprocess(df):
    df = clean_df(df)
    df = remove_stop_words(df)
    df = remove_punctuation(df) 
    df = remove_single_characters(df)
    df = df[df['READMES'].notna()]
    # df = stemming(df)
    df.to_csv("cleaned_df.csv", index=False)
    return df
    

In [76]:
df = preprocess(df)

/var/folders/ml/8_bkc7k91cb777tm6mxkvfbc0000gn/T/ipykernel_24469/2633761482.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df["READMES"] = df["READMES"].str.replace(r"[\"\',]", '')
/var/folders/ml/8_bkc7k91cb777tm6mxkvfbc0000gn/T/ipykernel_24469/2633761482.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['READMES'] = df['READMES'].str.replace('\d+', '')
/var/folders/ml/8_bkc7k91cb777tm6mxkvfbc0000gn/T/ipykernel_24469/1226715876.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["READMES"] = df['READMES'].str.replace('[^\w\s]','')


In [77]:
df = df[df['READMES'].notna()]
df.to_csv("cleaned_df.csv", index=False)

In [78]:
'''
def word_frequency(df):
    word_frequency_dict = {}
    for username,readme in zip(df["USERNAMES"], df["READMES"].str.lower()):
        word_frequency_dict[username] = {}
        for word in readme.split():
            if word not in word_frequency_dict[username].keys():
                word_frequency_dict[username][word] = 1
            else:
                word_frequency_dict[username][word] += 1
        word_frequency_dict[username] = (sorted(word_frequency_dict[username].items(), key=lambda x: -x[1]))
    return word_frequency_dict
'''


'\ndef word_frequency(df):\n    word_frequency_dict = {}\n    for username,readme in zip(df["USERNAMES"], df["READMES"].str.lower()):\n        word_frequency_dict[username] = {}\n        for word in readme.split():\n            if word not in word_frequency_dict[username].keys():\n                word_frequency_dict[username][word] = 1\n            else:\n                word_frequency_dict[username][word] += 1\n        word_frequency_dict[username] = (sorted(word_frequency_dict[username].items(), key=lambda x: -x[1]))\n    return word_frequency_dict\n'

In [79]:
df = pd.read_csv("cleaned_df.csv")
df = df[df['READMES'].notna()]
df.head(10)

,USERNAMES,READMES
0,alexbw,sampling inference bayesian hsmms hmms python ...
1,iassael,dearanking proposing hybrid dea polynomial int...
2,bshillingford,idtag fix codehackathon team members brendan s...
4,clementfarabet,sys moved community friendly repo xlua moved c...
5,ajabri,pytorch maml pytorch implementation supervised...
6,USERNAMES,readmes
7,coryf,homebrew personal homebrew formula sudoku curses
8,jrodrigomg,servidor nodejs proyecto sistemas operativos e...
9,vascofernandes,customersvrpf project consists two main parts ...
10,seba-1511,arduino chrono arduino based chronometer toosk...


In [80]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(df):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(df['READMES'])]

df['READMES'] = df.apply(lemmatize_text, axis=1)

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/Users/myasin/nltk_data'
    - '/opt/anaconda3/envs/thesis/nltk_data'
    - '/opt/anaconda3/envs/thesis/share/nltk_data'
    - '/opt/anaconda3/envs/thesis/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [81]:

vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0.01, stop_words='english')
vectors = vectorizer.fit_transform(df["READMES"])


In [82]:
feature_names = vectorizer.get_feature_names()

/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [83]:
termfreq = vectorizer.vocabulary_

In [84]:
query = "Computer vison"

In [85]:
query_vec = vectorizer.transform([query])

In [86]:
results = cosine_similarity(vectors,query_vec).reshape((-1,))

In [87]:
results.argsort()[-10:][::-1]

array([257, 220, 195, 329, 259, 338, 171, 150, 225, 148])

In [88]:
type(results)

numpy.ndarray

In [89]:
results.shape

(362,)

In [90]:
# Print Top 10 results
for i in results.argsort()[-10:][::-1]:
    print(df.iloc[i,0], results[i])


themadarchitect 0.07904647986752661
edgarriba 0.06879900769740685
intermilan 0.05401933393845209
rAm1n 0.05207870749500482
amirsaffari 0.05203149885115723
mirceamironenco 0.0487829589616814
SalemAmeen 0.04647568237169033
alxndrkalinin 0.04030317722675246
gr33ndata 0.03867786058278458
valthom 0.03691244395486756


In [91]:
# Print Top 10 results
for i in results.argsort()[-10:][::-1]:
    print(df.iloc[i,0], results[i])

themadarchitect 0.07904647986752661
edgarriba 0.06879900769740685
intermilan 0.05401933393845209
rAm1n 0.05207870749500482
amirsaffari 0.05203149885115723
mirceamironenco 0.0487829589616814
SalemAmeen 0.04647568237169033
alxndrkalinin 0.04030317722675246
gr33ndata 0.03867786058278458
valthom 0.03691244395486756


In [92]:
from pprint import pprint

In [106]:

recommended_users_plus_score = {}
query_vec = vectorizer.transform([query])
results = cosine_similarity(vectors,query_vec).reshape((-1,))
for name in results.argsort()[-10:][::-1]:
    recommended_users_plus_score[df.iloc[name,0]] = results[name]
print(recommended_users_plus_score)

{'themadarchitect': 0.07904647986752661, 'edgarriba': 0.06879900769740685, 'intermilan': 0.05401933393845209, 'rAm1n': 0.05207870749500482, 'amirsaffari': 0.05203149885115723, 'mirceamironenco': 0.0487829589616814, 'SalemAmeen': 0.04647568237169033, 'alxndrkalinin': 0.04030317722675246, 'gr33ndata': 0.03867786058278458, 'valthom': 0.03691244395486756}


In [109]:
def vectoriz_dataframe(df):

    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0.01, stop_words='english')
    vectorize_dataframe = vectorizer.fit_transform(df["READMES"])

    return vectorize_dataframe

In [111]:
def jobtitle_recommended(vectorize_dataframe, query):
    
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(vectorize_dataframe, query_vec).reshape((-1,))
    
    for i in results.argsort()[-10:][::-1]:
        print(df.iloc[i,0], results[i])
        
        recommended_users_plus_score = results[i]
    print(recommended_users_plus_score)   
    return recommended_users_plus_score

In [ ]:
score_per_user = jobtitle_recommended(vectorize_dataframe, query)